<a href="https://colab.research.google.com/github/manith506/projects/blob/main/hypothesis_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

### **Goal of this A/B test**

    * The goal of this A/B test is to analyze, whether there is a relationship between payment type and fare amount.

    for ex: discover whether the customers used to pay using credit card will pay higher fare amount than the customers used to pay by cash.



In [ ]:
df = pd.read_csv('/content/C2_2017_Yellow_Taxi_Trip_Data.csv.zip')
df.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,24870114,2,03/25/2017 8:55:43 AM,03/25/2017 9:09:47 AM,6,3.34,1,N,100,231,1,13.0,0.0,0.5,2.76,0.0,0.3,16.56
1,35634249,1,04/11/2017 2:53:28 PM,04/11/2017 3:19:58 PM,1,1.80,1,N,186,43,1,16.0,0.0,0.5,4.00,0.0,0.3,20.80
2,106203690,1,12/15/2017 7:26:56 AM,12/15/2017 7:34:08 AM,1,1.00,1,N,262,236,1,6.5,0.0,0.5,1.45,0.0,0.3,8.75
3,38942136,2,05/07/2017 1:17:59 PM,05/07/2017 1:48:14 PM,1,3.70,1,N,188,97,1,20.5,0.0,0.5,6.39,0.0,0.3,27.69
4,30841670,2,04/15/2017 11:32:20 PM,04/15/2017 11:49:03 PM,1,4.37,1,N,4,112,2,16.5,0.5,0.5,0.00,0.0,0.3,17.80


#### **payment types**

    when payment type = 1
        customers who are requested to pay using credit card.
    
    when payment type = 2
        customers who are requwsted to pay by cash.

### **Basic Exploratatry Data Analysis**

In [ ]:
df.columns

Index(['Unnamed: 0', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount'],
      dtype='object')

In [ ]:
''' df.info() will give the basic information about the dataset like datatypes of each column and
count of null values in each column.
fortunatly we don't have any null values in this dataset '''
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22699 entries, 0 to 22698
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             22699 non-null  int64  
 1   VendorID               22699 non-null  int64  
 2   tpep_pickup_datetime   22699 non-null  object 
 3   tpep_dropoff_datetime  22699 non-null  object 
 4   passenger_count        22699 non-null  int64  
 5   trip_distance          22699 non-null  float64
 6   RatecodeID             22699 non-null  int64  
 7   store_and_fwd_flag     22699 non-null  object 
 8   PULocationID           22699 non-null  int64  
 9   DOLocationID           22699 non-null  int64  
 10  payment_type           22699 non-null  int64  
 11  fare_amount            22699 non-null  float64
 12  extra                  22699 non-null  float64
 13  mta_tax                22699 non-null  float64
 14  tip_amount             22699 non-null  float64
 15  to

In [ ]:
''' if we have large number of columns to check which column containes null values we can use below code
here we don't have any columns with null values so we got empty list'''
[features for features in df.columns if df[features].isnull().sum() > 0]

[]

In [ ]:
'''  to check the count of null values in each column'''
df.isnull().sum()

Unnamed: 0               0
VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64

In [ ]:
df.describe()

,Unnamed: 0,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,2.269900e+04,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000
mean,5.675849e+07,1.556236,1.642319,2.913313,1.043394,162.412353,161.527997,1.336887,13.026629,0.333275,0.497445,1.835781,0.312542,0.299551,16.310502
std,3.274493e+07,0.496838,1.285231,3.653171,0.708391,66.633373,70.139691,0.496211,13.243791,0.463097,0.039465,2.800626,1.399212,0.015673,16.097295
min,1.212700e+04,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-120.000000,-1.000000,-0.500000,0.000000,0.000000,-0.300000,-120.300000
25%,2.852056e+07,1.000000,1.000000,0.990000,1.000000,114.000000,112.000000,1.000000,6.500000,0.000000,0.500000,0.000000,0.000000,0.300000,8.750000
50%,5.673150e+07,2.000000,1.000000,1.610000,1.000000,162.000000,162.000000,1.000000,9.500000,0.000000,0.500000,1.350000,0.000000,0.300000,11.800000
75%,8.537452e+07,2.000000,2.000000,3.060000,1.000000,233.000000,233.000000,2.000000,14.500000,0.500000,0.500000,2.450000,0.000000,0.300000,17.800000
max,1.134863e+08,2.000000,6.000000,33.960000,99.000000,265.000000,265.000000,4.000000,999.990000,4.500000,0.500000,200.000000,19.100000,0.300000,1200.290000


**Statistical analysis**

    by doing statistical analysis we get to know
    * the number of columns and their data types(either object or int64 or float64).
    * the count of null values in each columns.
    *most importantly we get to know 5 number summary
        (min, 25%, 50%, 75%, max) along with this we will get
        mean, standard deviation and count

In [ ]:
''' we are intrested in the relationship between payment type and fare amount that customers pay.
    one approch is to look out the average total fare amount for each payment type'''
avg_fare_amt = df.groupby(["payment_type"])["fare_amount"].mean()
avg_fare_amt

payment_type
1    13.429748
2    12.213546
3    12.186116
4     9.913043
Name: fare_amount, dtype: float64

    based on the average shown, it appears that customers who pay by credit card tends to pay a larger fare amount than the
    customers who pay by cash.
    However, this difference might arises from random sampling bais, rather than being a true diffrence in the total fare amount.
    to assess whether the difference is statistically significant, we conduct a hypothesis test.

## **Hypothesis Testing**

    before directly jumping into the hypothesis testing, we have to state the null hypothesis and alternative hypothesis.

    Null hypothesis(H_0) :
    There is no diffrence in the average total fare amount between customers who use credit card and customers who used pay by cash.

    Alternative Hypothesis(H_A) :            
    There is a diffrence in the average total fare amount between customers who use credit card and customers who used pay by cash.

    significance Level(α):
    for this experiment i have choosen 5% as the significance level and i proceed with the two sample t test.

In [ ]:
df_credit_card = df[df["payment_type" ]== 1]
df_cash = df[df["payment_type" ]== 2]
stats.ttest_ind(df_credit_card["fare_amount"], df_cash["fare_amount"], equal_var= False, alternative="two-sided")

TtestResult(statistic=6.866800855655372, pvalue=6.797387473030518e-12, df=16675.48547403633)

    since the p-value is less than the 0.05(p-value<0.05), we can reject the null hypothesis,
    that means there is a significant difference between the average total fare amount between customers who uses credit card
    and customers who used to pay by cash.

    (Extra) Hypothesis testing :           
    by the above hypothesis testing we got that, there is a diffrence in the average fare amount between customers pay using credit cards
    and by cash.
    but, i want to get to know whether credit card customers will pay higher or lower fare amount than the customers who pay by cash
    for that we will conduct another hypothesis testing

    Null hypothesis(H_0) :
    The customers who use credit cards pay lower or equal fare amounts than customers who use cash.

    Alternative Hypothesis(H_A) :            
    The customers who use credit cards pay higher fare amounts than customers who use cash.

    significance Level(α):
    for this experiment i have choosen 5% as the significance level and i proceed with the two sample t test.


In [ ]:
stats.ttest_ind(df_credit_card["fare_amount"], df_cash["fare_amount"], alternative="greater")

TtestResult(statistic=6.469087519667181, pvalue=5.0310632075178034e-11, df=22530.0)

    since the p-value is less than the 0.05(p-value<0.05), we can reject the null hypothesis, in other words we fail
    to reject the alternative hypothesis
    that means, The customers who use credit cards pay higher fare amounts than customers who use cash.


### **Insights from the hypothesis testing**

    1) through our hypothesis tests, we can conclude that there is a significant diffrence in  average fare amount between customers
    pay using credit cards and by cash also also the customers who pay using credit card will pay higher fare amount than the customers
    who pay by cash.

    2)In this case we can setup some ways to encourage customers to pay using credit cards more than cash. foe ex:
        * set up a rewards for credit card users only.
        * show commercial(s) about credit cards and there rewards in the taxi, so the customers will see the commercials
           when they take the taxi.
        * Design a web app, we can offer the special benifits for customers who uses a web app to book the service,
           it will force the customers to use a credit cards.